# Segmenting and Clustering Neighborhoods in Toronto

## Step 1 : Data Gathering from Wikipedia

In [98]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 

In [99]:
page=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup=BeautifulSoup(page.content, 'html.parser')


In [100]:
table_class='mw-parser-output'
items=soup.find(class_='mw-parser-output')
table_class=items.find(class_='wikitable sortable')
required_data=tabled.find_all('tr')

#### Creating DataFrame

In [137]:
#column_names = ['Postal-code','Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame()

In [138]:
for i in range(1,len(required_data)):
    td=required_data[i].find_all('td')
    postal_code=td[0].get_text().strip()
    borough=td[1].get_text().strip()
    neighbor=td[2].get_text().strip()
    
    neighborhoods=neighborhoods.append({'Postel-code':postal_code,
                         'Borough':borough,
                         'Neighborhood':neighbor}, ignore_index=True)

In [139]:
neighborhoods=neighborhoods[['Postel-code', 'Borough','Neighborhood']]

In [140]:
neighborhoods.shape

(180, 3)

## Step 2 Data Cleanup

#### 2.1 : removing rows with Not Assigned Borough

In [141]:
neighborhoods.drop(neighborhoods[neighborhoods.Borough=='Not assigned'].index, inplace=True)


#### 2.2 : More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [142]:
neighborhoods["Neighborhood"] =neighborhoods.groupby("Postel-code")["Neighborhood"].transform(lambda neigh: ', '.join(neigh))

#### 2.3 : dropping duplicates

In [144]:
neighborhoods=neighborhoods.drop_duplicates()

#### 2.4 : If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [147]:
neighborhoods['Neighborhood'].replace("Not assigned", neighborhoods["Borough"],inplace=True)

In [154]:
#updating the index 
neighborhoods=neighborhoods.reset_index(drop=True)

In [155]:
neighborhoods

,Postel-code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Step 3 : Data Frame Shape Output

In [156]:
neighborhoods.shape

(103, 3)